In [1]:
import pandas as pd
import glob

# Find all your result CSV files
all_files = glob.glob('results_*.csv')

# Create a list to hold each DataFrame
li = []

# Loop through the files, read them, and add platform/test info
for filename in all_files:
    df = pd.read_csv(filename)
    # Extract info from the filename
    parts = filename.replace('.csv', '').split('_')
    df['platform'] = parts[1].upper()
    df['test_type'] = parts[2].upper()
    df['payload_size'] = parts[3]
    li.append(df)

# Combine all DataFrames into one
master_df = pd.concat(li, axis=0, ignore_index=True)

# Display the first few rows to confirm it worked
print("Successfully loaded and combined all data!")
master_df.head()

ValueError: No objects to concatenate